# 유사한 단어찾기 게임
1. 사전 학습된 모델 또는 적절한 데이터셋을 찾는다.
2. 워드 임베딩 모델으 학습시칸다
3. 단어유사도가 0.8 이상인  a,b를 랜덤 추출한다
4. A,B와 대응되는 C를 추출한다
5. D를 입력받는다 
=> 
A:B = C:D관계에 대응하는 D를 찾는 게임을 만든다 
ex) A:산, B:바다, C:나무, D:물

**출력예시**

관계 [ 수긍 : 추락 = 대사관 : ? ]
모델이 예측한 가장 적합한 단어 : 잠입
당신의 답변과 모델 예측 유사도 : 0.34
아쉽네요. 더 생각해보세요

**출력예시**

관계 [ 수긍 : 추락 = 대사관 : ? ]
모델이 예측한 가장 적합한 단어 : 잠입
당신의 답변과 모델 예측 유사도 : 0.34
아쉽네요. 더 생각해보세요


In [ ]:
import pandas as pd
from gensim.models import Word2Vec
import re
from konlpy.tag import Okt

data = pd.read_csv('naver_movie_ratings.txt', sep='\t')
document = data['document']

okt = Okt()
def preprocess(text):

    if pd.isna(text):
        return []

    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]', '', str(text))

    return okt.nouns(text)


preprocessed_sentences = document.apply(preprocess).tolist()

preprocessed_sentences = [s for s in preprocessed_sentences if s]

print(f"전처리된 문장 개수: {len(preprocessed_sentences)}")
print(f"전처리된 첫 번째 문장: {preprocessed_sentences[0]}")

전처리된 문장 개수: 192122
전처리된 첫 번째 문장: ['때', '보고', '지금', '다시']


In [ ]:
# Word2Vec 모델 학습
model = Word2Vec(
    sentences = preprocessed_sentences,
    vector_size = 100,  
    window = 5,      
    min_count = 5,      
    sg = 0             
)


print(f"모델 벡터의 크기: {model.wv.vectors.shape}")

모델 벡터의 크기: (13741, 100)


In [ ]:
# import random

# def find_similar_words(model, similarity_threshold=0.8):
#     all_words = list(model.wv.index_to_key)
#     while True:
#         a = random.choice(all_words)
#         b = random.choice(all_words)
#         if a == b:
#             continue
#         try:
#             similarity = model.wv.similarity(a, b)
#             if similarity > similarity_threshold:
#                 return a, b
#         except KeyError:
#             continue

import random

def find_similar_words(model, similarity_threshold=0.8):
    """유사도 임계값 이상의 단어 쌍 (A, B)을 찾습니다."""
    all_words = list(model.wv.index_to_key)
    
    while True:
        a = random.choice(all_words)
        b = random.choice(all_words)
        if a == b:
            continue
            
        try:
            similarity = model.wv.similarity(a, b)

            if similarity > similarity_threshold:
                return a, b
        except KeyError:

            continue


word_a, word_b = find_similar_words(model, similarity_threshold=0.8)
print(f"찾은 유사 단어 쌍: {word_a}, {word_b}")

찾은 유사 단어 쌍: 현상, 조심


In [ ]:
def find_unrelated_word(model, word_a, word_b):
    """A, B와 관계가 없는 단어 C를 찾습니다."""
    all_words = list(model.wv.index_to_key)
    
    while True:
        c = random.choice(all_words)

        if c not in [word_a, word_b]:
            return c


word_c = find_unrelated_word(model, word_a, word_b)
print(f"찾은 관계 단어: {word_c}")

찾은 관계 단어: 빠삐용


In [ ]:

# !pip install pandas gensim konlpy

import pandas as pd
from gensim.models import Word2Vec
import re
from konlpy.tag import Okt
import random


print("1. 데이터셋을 로드하고 모델을 학습")

try:
    data = pd.read_csv('naver_movie_ratings.txt', sep='\t')
    document = data['document']
except FileNotFoundError:
    print("오류-파일을 찾을 수 없음.  파일 경로를 확인해야함")
    exit()


okt = Okt()


def preprocess(text):
    if pd.isna(text):
        return []

    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]', '', str(text))

    return okt.nouns(text)


preprocessed_sentences = document.apply(preprocess).tolist()
preprocessed_sentences = [s for s in preprocessed_sentences if s]

print("   - 전처리 완료.")


model = Word2Vec(
    sentences=preprocessed_sentences,
    vector_size=100,
    window=5,
    min_count=5,
    sg=0,
    workers=4
)
print("   - Word2Vec 모델 학습 완료.")
print("-" * 30)


def find_similar_words(model, similarity_threshold=0.7):
    all_words = list(model.wv.index_to_key)
    while True:
        a = random.choice(all_words)
        try:

            similar_candidates = [
                (w, sim) for w, sim in model.wv.most_similar(a, topn=30)
                if sim > similarity_threshold
            ]
            if similar_candidates:
                b, sim = random.choice(similar_candidates)
                return a, b
        except KeyError:
            continue


def find_unrelated_word(model, word_a, word_b):
    """A, B와 관련 없는 단어 C를 무작위로 찾습니다."""
    all_words = list(model.wv.index_to_key)
    while True:
        c = random.choice(all_words)
        if c not in [word_a, word_b]:
            return c

def find_analogy(model, a, b, c):
    """A:B = C:D 관계에서 D를 예측합니다."""

    result = model.wv.most_similar(positive=[c, b], negative=[a], topn=1)
    if result:
        return result[0][0], result[0][1] # (단어, 유사도)
    return None, 0


print("2. 유사 단어 찾기 게임 시작!")
print("게임을 종료하려면 '종료'를 입력하세요.")
print("-" * 30)

while True:
    # 3단계: 유사 단어 쌍 (A, B) 추출
    word_a, word_b = find_similar_words(model, similarity_threshold=0.7)
    
    # 4단계: 관계에 대응하는 단어 (C) 추출
    word_c = find_unrelated_word(model, word_a, word_b)

    # 모델이 예측한 가장 적합한 단어 (정답) 찾기
    model_d, model_similarity = find_analogy(model, word_a, word_b, word_c)

    print(f"관계 [ {word_a} : {word_b} = {word_c} : ? ]")
    user_d = input("답변은? ")

    if user_d == '종료':
        print("게임을 종료합니다.")
        break
    
    try:
        user_similarity = model.wv.similarity(user_d, model_d)
        

        print(f"모델이 예측한 가장 적합한 단어 : {model_d}")
        print(f"당신의 답변과 모델 예측 유사도 : {user_similarity:.2f}")
        
        if user_similarity > 0.7: # 정답 유사도 기준
            print("정답입니다.")
        else:
            print("아쉽네요. 더 생각해보세요.")
        
    except KeyError:
        print("입력한 단어가 모델에 없습니다. 다시 시도해주세요.")

    print("-" * 30)

1. 데이터셋을 로드하고 모델을 학습시키는 중...
   - 전처리 완료.
   - Word2Vec 모델 학습 완료.
------------------------------
2. 유사 단어 찾기 게임 시작!
게임을 종료하려면 '종료'를 입력하세요.
------------------------------
관계 [ 정자 : 돼지고기 = 토마스 : ? ]
모델이 예측한 가장 적합한 단어 : 고요한
당신의 답변과 모델 예측 유사도 : 0.87
정답입니다.
------------------------------
관계 [ 공연장 : 저녁 = 부정 : ? ]
모델이 예측한 가장 적합한 단어 : 누군가
당신의 답변과 모델 예측 유사도 : 0.80
정답입니다.
------------------------------
관계 [ 박재정 : 격감 = 차라 : ? ]
모델이 예측한 가장 적합한 단어 : 칼부림
당신의 답변과 모델 예측 유사도 : 0.55
아쉽네요. 더 생각해보세요.
------------------------------
관계 [ 미미 : 만호 = 감각 : ? ]
모델이 예측한 가장 적합한 단어 : 디테일
당신의 답변과 모델 예측 유사도 : 0.62
정답입니다.
------------------------------
관계 [ 실험영화 : 짤 = 트럭 : ? ]
입력한 단어가 모델에 없습니다. 다시 시도해주세요.
------------------------------
관계 [ 코비 : 박하 = 위화 : ? ]
모델이 예측한 가장 적합한 단어 : 정선
당신의 답변과 모델 예측 유사도 : 0.57
아쉽네요. 더 생각해보세요.
------------------------------
관계 [ 인력 : 형상 = 껄껄 : ? ]
모델이 예측한 가장 적합한 단어 : 유민
당신의 답변과 모델 예측 유사도 : 0.89
정답입니다.
------------------------------
관계 [ 엔돌핀 : 최정원 = 윌리엄스 : ? ]
입력한 단어가 모델에 없습니다. 다시 시도해주세요.
-

In [ ]:

# !pip install pandas gensim konlpy

import pandas as pd
from gensim.models import Word2Vec
import re
from konlpy.tag import Okt
import random


print("1. 데이터셋을 로드하고 모델을 학습시키는 중...")

try:
    data = pd.read_csv('naver_movie_ratings.txt', sep='\t')
    document = data['document']
except FileNotFoundError:
    print("오류: 'naver_movie_ratings.txt' 파일을 찾을 수 없습니다. 파일 경로를 확인해주세요.")
    exit()


okt = Okt()

# 전처리 함수
def preprocess(text):
    if pd.isna(text):
        return []
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]', '', str(text))

    return okt.nouns(text)


preprocessed_sentences = document.apply(preprocess).tolist()
preprocessed_sentences = [s for s in preprocessed_sentences if s]

print(f"   - 전처리 완료. 전처리된 문장 개수: {len(preprocessed_sentences)}")
print(f"   - 전처리된 첫 번째 문장: {preprocessed_sentences[0]}")


model = Word2Vec(
    sentences=preprocessed_sentences,
    vector_size=100,
    window=5,
    min_count=5,
    sg=0,
    workers=4
)
print(f"   - Word2Vec 모델 학습 완료. 모델 벡터의 크기: {model.wv.vectors.shape}")
print("-" * 30)




def find_similar_words(model, similarity_threshold=0.7):
    """Word2Vec most_similar을 이용해 더 자연스러운 단어 쌍을 찾음"""
    all_words = list(model.wv.index_to_key)
    while True:
        a = random.choice(all_words)
        try:

            similar_candidates = [
                (w, sim) for w, sim in model.wv.most_similar(a, topn=30)
                if sim > similarity_threshold
            ]
            if similar_candidates:
                b, sim = random.choice(similar_candidates)
                return a, b
        except KeyError:
            continue

def find_unrelated_word(model, word_a, word_b):
    """A, B와 관련 없는 단어 C를 무작위로 찾습니다."""
    all_words = list(model.wv.index_to_key)
    while True:
        c = random.choice(all_words)
        if c not in [word_a, word_b]:
            return c

def find_analogy(model, a, b, c):
    """A:B = C:D 관계에서 D를 예측합니다."""
    result = model.wv.most_similar(positive=[c, b], negative=[a], topn=1)
    if result:
        return result[0][0], result[0][1]  # (단어, 유사도)
    return None, 0



print("2. 유사 단어 찾기 게임 시작!")
print("게임을 종료하려면 '종료'를 입력하세요.")
print("-" * 30)

while True:
    # 3단계: 유사 단어 쌍 (A, B) 추출
    word_a, word_b = find_similar_words(model, similarity_threshold=0.7)

    # 4단계: 관계에 대응하는 단어 (C) 추출
    word_c = find_unrelated_word(model, word_a, word_b)

    # 모델이 예측한 가장 적합한 단어 (정답) 찾기
    model_d, model_similarity = find_analogy(model, word_a, word_b, word_c)

    # 사용자 입력 받기
    print(f"관계 [ {word_a} : {word_b} = {word_c} : ? ]")
    user_d = input("당신의 답변은? ")

    if user_d == '종료':
        print("게임을 종료합니다.")
        break


    try:
        user_similarity = model.wv.similarity(user_d, model_d)

        print(f"모델이 예측한 가장 적합한 단어 : {model_d}")
        print(f"당신의 답변과 모델 예측 유사도 : {user_similarity:.2f}")

        if user_similarity > 0.7:
            print("정답입니다.")
        else:
            print("아쉽네요. 더 생각해보세요.")

    except KeyError:
        print("입력한 단어가 모델에 없습니다. 다시 시도해주세요.")

    print("-" * 30)


1. 데이터셋을 로드하고 모델을 학습시키는 중...
   - 전처리 완료. 전처리된 문장 개수: 192122
   - 전처리된 첫 번째 문장: ['때', '보고', '지금', '다시']
   - Word2Vec 모델 학습 완료. 모델 벡터의 크기: (13741, 100)
------------------------------
2. 유사 단어 찾기 게임 시작!
게임을 종료하려면 '종료'를 입력하세요.
------------------------------
관계 [ 강마 : 마스크 = 미녀삼총사 : ? ]
게임을 종료합니다.


In [ ]:

# # !pip install pandas gensim konlpy beautifulsoup4

# import pandas as pd
# from gensim.models import Word2Vec
# import re
# from konlpy.tag import Okt
# import random
# import glob
# from bs4 import BeautifulSoup
# import numpy as np
# import sys

# random.seed(42)


# print("1. 데이터셋을 로드하고 모델을 학습시키는 중...")


# def parse_xml_files(file_paths):
#     sentences = []
#     for file_path in file_paths:
#         try:
#             with open(file_path, 'r', encoding='utf-8') as f:
#                 xml_data = f.read()
#         except Exception as e:
#             print(f"[경고] {file_path} 읽기 실패: {e}")
#             continue

#         soup = BeautifulSoup(xml_data, 'xml')
#         lexical_entries = soup.find_all('LexicalEntry')
        
#         for entry in lexical_entries:
#             word_tag = entry.find('Lemma')
#             if not word_tag:
#                 continue
#             feat_tag = word_tag.find('feat', {'att': 'writtenForm'})
#             if not feat_tag or not feat_tag.has_attr('val'):
#                 continue
#             word = feat_tag['val']

#             definitions = []
#             for s in entry.find_all('Sense'):
#                 feat = s.find('feat', {'att': 'definition'})
#                 if feat and feat.has_attr('val'):
#                     definitions.append(feat['val'])

#             if definitions:
#                 sentence = word + ' ' + ' '.join(definitions)
#                 sentences.append(sentence)
#     return sentences


# try:
#     xml_files = glob.glob("*.xml")
#     if not xml_files:
#         raise FileNotFoundError
#     raw_sentences = parse_xml_files(xml_files)
# except FileNotFoundError:
#     print("오류: 현재 디렉터리에서 XML 파일을 찾을 수 없습니다.")
#     sys.exit(0)

# okt = Okt()

# def preprocess(text):
#     if pd.isna(text) or not isinstance(text, str):
#         return []

#     text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]', ' ', text)

#     nouns = [n for n in okt.nouns(text) if len(n) >= 2]
#     return nouns

# preprocessed_sentences = [preprocess(s) for s in raw_sentences]
# preprocessed_sentences = [s for s in preprocessed_sentences if s]

# if not preprocessed_sentences:
#     print("전처리된 문장이 없습니다. XML 데이터를 확인하세요.")
#     sys.exit(0)

# print(f"    - 전처리 완료. 전처리된 문장 개수: {len(preprocessed_sentences)}")
# print(f"    - 전처리된 첫 번째 문장: {preprocessed_sentences[0][:15]} ...")

# min_count = 5 if len(preprocessed_sentences) > 500 else 2
# model = Word2Vec(
#     sentences=preprocessed_sentences,
#     vector_size=200,
#     window=5,
#     min_count=min_count,
#     sg=0,
#     workers=4
# )
# wv = model.wv
# print(f"    - Word2Vec 학습 완료. 어휘 수: {len(wv.index_to_key)} | 벡터 차원: {wv.vectors.shape[1]}")
# print(f"    - 사용된 min_count 값: {min_count}")
# print("-" * 30)

# def is_valid_token(token):
#     if token is None:
#         return False
#     if token not in wv:
#         return False

#     if any(ch.isdigit() for ch in token):
#         return False
#     if len(token) >= 10:
#         return False

#     try:
#         cnt = wv.get_vecattr(token, "count")
#         if cnt < max(min_count, 3):
#             return False
#     except Exception:
#         pass
#     return True

# def cosine(u, v):
#     un = np.linalg.norm(u); vn = np.linalg.norm(v)
#     if un == 0 or vn == 0:
#         return 0.0
#     return float(np.dot(u, v) / (un * vn))


# def pick_similar_pair(sim_min=0.45, sim_max=0.92, max_trials=200):
#     """
#     A를 뽑고, A와 '적당히' 유사한 B를 고른다.
#     너무 가까운 동의어/중복은 피하고, 너무 먼 관계도 피함.
#     """
#     vocab = [w for w in wv.index_to_key if is_valid_token(w)]
#     for _ in range(max_trials):
#         a = random.choice(vocab)
#         try:
#             cands = [(w, s) for w, s in wv.most_similar(a, topn=40)
#                      if is_valid_token(w) and sim_min <= s <= sim_max and w != a]
#             if not cands:
#                 continue
#             b = random.choice(cands)[0]
#             return a, b
#         except KeyError:
#             continue
#     return None, None

# def pick_c_close_to_a(a, b, sim_a_min=0.35, sim_a_max=0.85, sim_b_max=0.55, max_trials=200):
#     """
#     C는 A와 '같은 범주'로 보이도록 A와 어느 정도 비슷하게 고른다.
#     단, B와는 너무 가깝지 않게 하여 삼항이 망가지지 않도록 함.
#     """
#     try:
#         neighbors = [w for w, s in wv.most_similar(a, topn=60)]
#     except KeyError:
#         return None
#     random.shuffle(neighbors)
#     for c in neighbors:
#         if c in (a, b) or not is_valid_token(c):
#             continue
#         try:
#             sim_ac = wv.similarity(a, c)
#             sim_bc = wv.similarity(b, c)
#         except KeyError:
#             continue
#         if sim_a_min <= sim_ac <= sim_a_max and sim_bc <= sim_b_max:
#             return c

#     vocab = [w for w in wv.index_to_key if is_valid_token(w)]
#     random.shuffle(vocab)
#     for c in vocab[:200]:
#         if c in (a, b):
#             continue
#         try:
#             sim_ac = wv.similarity(a, c)
#             sim_bc = wv.similarity(b, c)
#         except KeyError:
#             continue
#         if sim_a_min <= sim_ac <= sim_a_max and sim_bc <= sim_b_max:
#             return c
#     return None

# def solve_analogy(a, b, c, topn=15):
#     """
#     3CosMul로 D 후보를 구하고, 관계 일관성 점수로 필터링.
#     관계 일관성 = cos( (b-a), (d-c) )
#     """
#     if not all(t in wv for t in [a, b, c]):
#         return None, 0.0, []
#     try:
#         # 3CosMul 사용
#         cand_list = wv.most_similar_cosmul(positive=[b, c], negative=[a], topn=topn+5)
#     except Exception:
#         # gensim 버전에 따라 cosmul 미지원 시 fallback
#         cand_list = wv.most_similar(positive=[b, c], negative=[a], topn=topn+5)

#     av, bv, cv = wv[a], wv[b], wv[c]
#     rel_vec = bv - av
#     filtered = []
#     for d, base_score in cand_list:
#         if d in (a, b, c) or not is_valid_token(d):
#             continue
#         score_rel = cosine(rel_vec, wv[d] - cv)

#         if score_rel >= 0.30:
#             filtered.append((d, base_score, score_rel))
#         if len(filtered) >= topn:
#             break

#     if not filtered:
#         return None, 0.0, []


#     filtered.sort(key=lambda x: (x[2], x[1]), reverse=True)
#     d, base_score, rel_score = filtered[0]
#     return d, rel_score, filtered[:5]  # 상위 5개 후보 반환


# print("2. 유사 단어(아날로지) 게임 시작!")
# print("게임을 종료하려면 '종료'를 입력하세요.")
# print("-" * 30)

# while True:
#     try:
#         # 문제 생성 (여러 번 시도 -> 그럴듯한 삼항으로 )
#         ok = False
#         for _ in range(30):
#             a, b = pick_similar_pair()
#             if not a or not b:
#                 continue
#             c = pick_c_close_to_a(a, b)
#             if not c:
#                 continue
#             d, rel_score, top5 = solve_analogy(a, b, c, topn=10)
#             if d and rel_score >= 0.32:
#                 ok = True
#                 break
#         if not ok:
#             print("적절한 문제를 만들기 어려워 다음 문제로 넘어갑니다.")
#             print("-" * 30)
#             continue

#         print(f"관계 [ {a} : {b} = {c} : ? ]")
#         user_d = input("당신의 답변은? ").strip()
#         if user_d == '종료':
#             print("게임을 종료합니다.")
#             break

#         # 모델 예측 및 피드백
#         print(f"모델이 예측한 가장 적합한 단어 : {d} (관계 일관성: {rel_score:.2f})")
#         try:
#             user_sim = wv.similarity(user_d, d)
#             print(f"당신의 답변과 모델 예측 유사도 : {user_sim:.2f}")
#             if user_d == d or user_sim >= 0.70:
#                 print("정답입니다.")
#             elif user_sim >= 0.50:
#                 print("아주 근접")
#             else:
#                 print("아쉽네요. 다음 문제 ")
#         except KeyError:
#             print(f"입력한 단어 '{user_d}'가 모델 어휘에 없습니다. (철자/형태를 확인해보세요)")


#     except (IndexError, AttributeError, KeyError) as e:
#         print(f"[경고] 예외 발생: {e}. 다음 문제로 넘어갑니다.")
#         print("-" * 30)
#         continue


1. 데이터셋을 로드하고 모델을 학습시키는 중...
    - 전처리 완료. 전처리된 문장 개수: 31986
    - 전처리된 첫 번째 문장: ['눈엣', '가시', '몹시', '보기', '사람'] ...
    - Word2Vec 학습 완료. 어휘 수: 8076 | 벡터 차원: 200
    - 사용된 min_count 값: 5
------------------------------
2. 유사 단어(아날로지) 게임 시작!
게임을 종료하려면 '종료'를 입력하세요.
------------------------------
관계 [ 실무 : 목별 = 앞서 : ? ]
모델이 예측한 가장 적합한 단어 : 지점 (관계 일관성: 0.51)
입력한 단어 '나중에'가 모델 어휘에 없습니다. (철자/형태를 확인해보세요)
참고 후보: 지점(rel 0.51), 도착(rel 0.50), 수비(rel 0.42), 좌우(rel 0.41), 외야(rel 0.34)
------------------------------
관계 [ 구세주 : 왕후 = 공동체 : ? ]
모델이 예측한 가장 적합한 단어 : 비판 (관계 일관성: 0.65)
입력한 단어 '단합'가 모델 어휘에 없습니다. (철자/형태를 확인해보세요)
참고 후보: 비판(rel 0.65), 취직(rel 0.61), 혜택(rel 0.57), 소득(rel 0.56), 무질서(rel 0.49)
------------------------------
관계 [ 적삼 : 웃옷 = 사이 : ? ]
모델이 예측한 가장 적합한 단어 : 공간 (관계 일관성: 0.46)
당신의 답변과 모델 예측 유사도 : 0.26
아쉽네요. 다음 문제로 가볼까요?
참고 후보: 공간(rel 0.46), 멱살(rel 0.36), 도우(rel 0.36), 깜박(rel 0.35), 관계도(rel 0.34)
------------------------------
관계 [ 바닷물고기 : 매실 = 출전 : ? ]
모델이 예측한 가장 적합한 단어 : 최저 (관계 일관성: 0.39)
당

KeyboardInterrupt: Interrupted by user

In [5]:

# !pip install pandas gensim konlpy beautifulsoup4

import pandas as pd
from gensim.models import Word2Vec
import re
from konlpy.tag import Okt
import random
import glob
from bs4 import BeautifulSoup

print("1. 데이터셋을 로드하고 모델을 학습시키는 중...")


def parse_xml_files(file_paths):
    sentences = []
    for file_path in file_paths:
        with open(file_path, 'r', encoding='utf-8') as f:
            xml_data = f.read()
        
        soup = BeautifulSoup(xml_data, 'xml')
        lexical_entries = soup.find_all('LexicalEntry')
        
        for entry in lexical_entries:

            word_tag = entry.find('Lemma')
            if word_tag:
                word = word_tag.find('feat', {'att': 'writtenForm'})['val']
            else:
                continue


            definitions = [
                s.find('feat', {'att': 'definition'})['val'] 
                for s in entry.find_all('Sense') 
                if s.find('feat', {'att': 'definition'})
            ]
            
            if definitions:
                sentence = word + ' ' + ' '.join(definitions)
                sentences.append(sentence)
                
    return sentences


try:
    xml_files = glob.glob("*.xml")
    if not xml_files:
        raise FileNotFoundError
    

    raw_sentences = parse_xml_files(xml_files)
    
except FileNotFoundError:
    print("오류: 현재 디렉터리에서 XML 파일을 찾을 수 없습니다.")
    exit()

okt = Okt()


def preprocess(text):
    if pd.isna(text) or not isinstance(text, str):
        return []
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]', '', text)
    return okt.nouns(text)


preprocessed_sentences = [preprocess(s) for s in raw_sentences]
preprocessed_sentences = [s for s in preprocessed_sentences if s]

print(f"    - 전처리 완료. 전처리된 문장 개수: {len(preprocessed_sentences)}")
print(f"    - 전처리된 첫 번째 문장: {preprocessed_sentences[0]}")

model = Word2Vec(
    sentences=preprocessed_sentences,
    vector_size=100,
    window=5,
    min_count=5,
    sg=0,
    workers=4
)
print(f"    - Word2Vec 모델 학습 완료. 모델 벡터의 크기: {model.wv.vectors.shape}")
print("-" * 30)


def find_similar_words(model, similarity_threshold=0.5):
    """Word2Vec most_similar을 이용해 더 자연스러운 단어 쌍을 찾음"""
    all_words = list(model.wv.index_to_key)
    while True:
        a = random.choice(all_words)
        try:
            similar_candidates = [
                (w, sim) for w, sim in model.wv.most_similar(a, topn=30)
                if sim > similarity_threshold
            ]
            if similar_candidates:
                b, sim = random.choice(similar_candidates)
                return a, b
        except KeyError:
            continue

def find_unrelated_word(model, word_a, word_b):
    """A, B와 관련 없는 단어 C를 무작위로 찾습니다. A, B와는 유의미한 거리를 가집니다."""
    all_words = list(model.wv.index_to_key)
    while True:
        c = random.choice(all_words)

        if c not in [word_a, word_b]:
            try:

                if model.wv.similarity(word_a, c) < 0.3 and model.wv.similarity(word_b, c) < 0.3:
                    return c
            except KeyError:
                continue

def find_analogy(model, a, b, c):
    """A:B = C:D 관계에서 D를 예측합니다."""
    if not all(word in model.wv for word in [a, b, c]):
        return None, 0
    
    result = model.wv.most_similar(positive=[c, b], negative=[a], topn=1)
    if result:
        return result[0][0], result[0][1]
    return None, 0


print("2. 유사 단어 찾기 게임 시작!")
print("게임을 종료하려면 '종료'를 입력하세요.")
print("-" * 30)

while True:
    try:
        word_a, word_b = find_similar_words(model)
        word_c = find_unrelated_word(model, word_a, word_b)
        
        if not word_c:
            print("적절한 유추 단어를 찾을 수 없습니다. 다음 문제로 넘어갑니다.")
            print("-" * 30)
            continue

        model_d, model_similarity = find_analogy(model, word_a, word_b, word_c)

        if not model_d:
            print("적절한 유추 단어를 찾을 수 없습니다. 다음 문제로 넘어갑니다.")
            print("-" * 30)
            continue
            
        print(f"관계 [ {word_a} : {word_b} = {word_c} : ? ]")
        user_d = input("당신의 답변은? ")

        if user_d == '종료':
            print("게임을 종료합니다.")
            break

        try:
            user_similarity = model.wv.similarity(user_d, model_d)

            print(f"모델이 예측한 가장 적합한 단어 : {model_d}")
            print(f"당신의 답변과 모델 예측 유사도 : {user_similarity:.2f}")

            if user_similarity > 0.7:
                print("정답입니다.")
            else:
                print("아쉽네요. 더 생각해보세요.")

        except KeyError:
            print(f"입력한 단어 '{user_d}'가 모델에 없습니다. 다시 시도해주세요.")

        print("-" * 30)
        
    except (IndexError, AttributeError) as e:
        print(f"오류 발생: {e}. 게임을 다시 시작합니다.")
        print("-" * 30)
        continue

1. 데이터셋을 로드하고 모델을 학습시키는 중...
    - 전처리 완료. 전처리된 문장 개수: 32127
    - 전처리된 첫 번째 문장: ['눈엣', '가시', '몹시', '보기', '사람']
    - Word2Vec 모델 학습 완료. 모델 벡터의 크기: (8655, 100)
------------------------------
2. 유사 단어 찾기 게임 시작!
게임을 종료하려면 '종료'를 입력하세요.
------------------------------
관계 [ 옛날 : 신선 = 화이트보드 : ? ]
모델이 예측한 가장 적합한 단어 : 나사
당신의 답변과 모델 예측 유사도 : 0.81
정답입니다.
------------------------------
관계 [ 개선 : 첨단 = 컴퍼스 : ? ]
모델이 예측한 가장 적합한 단어 : 번갈아
당신의 답변과 모델 예측 유사도 : 0.24
아쉽네요. 더 생각해보세요.
------------------------------
관계 [ 타당성 : 마태복음 = 명단 : ? ]
모델이 예측한 가장 적합한 단어 : 구성원
당신의 답변과 모델 예측 유사도 : 0.30
아쉽네요. 더 생각해보세요.
------------------------------
관계 [ 스크린 : 상표 = 양 : ? ]
모델이 예측한 가장 적합한 단어 : 염소
당신의 답변과 모델 예측 유사도 : 0.70
정답입니다.
------------------------------
관계 [ 이유나 : 부림 = 부분 : ? ]
모델이 예측한 가장 적합한 단어 : 아래쪽
당신의 답변과 모델 예측 유사도 : 0.46
아쉽네요. 더 생각해보세요.
------------------------------
관계 [ 협력 : 조언 = 인용 : ? ]
모델이 예측한 가장 적합한 단어 : 부호
당신의 답변과 모델 예측 유사도 : 0.25
아쉽네요. 더 생각해보세요.
------------------------------
관계 [ 시간대 : 일정 = 리다 : ? ]
게임을 